In [1]:
!pip install pandas
!pip install sklearn
!pip install scipy
!pip install numpy
!pip install pandas
!pip install joblib
!pip install matplotlib
!pip install seaborn

In [2]:
#Loading of the initial data file
import pandas as pd
data_file = 'synthetic_efdata.csv'
df = pd.read_csv(data_file)

# Data Cleaning

In [3]:
#Missing values
df.fillna(df.mean(), inplace =True)

#duplicates
df.drop_duplicates(inplace = True)

In [4]:
#check no duplicates/missing values
print("Duplicate Rows:", df.duplicated().sum())
print("Missing Values:", df.isnull().sum())

Duplicate Rows: 0
Missing Values: Income ($)                        0
Monthly Expenses ($)              0
Dependents                        0
Job Stability (1-10)              0
Recommended Emergency Fund ($)    0
dtype: int64


In [5]:
#data type conversion
df['Income ($)'] = df['Income ($)'].astype(int)
df['Monthly Expenses ($)'] = df['Monthly Expenses ($)'].astype(int)
df['Dependents'] = df['Dependents'].astype(int)
df['Job Stability (1-10)'] = df['Job Stability (1-10)'].astype(int)
df['Lifestyle Cost (1-10)'] = df['Lifestyle Cost (1-10)'].astype(int)
df['Recommended Emergency Fund ($)'] = df['Recommended Emergency Fund ($)'].astype(int)

In [6]:
#Remomve outliers (currently at threshold of 3 standard deviation)
import numpy as np
from scipy import stats

threshold = 3

z_scores = np.abs(stats.zscore(df.select_dtypes(include=np.number))) #only number variables are selected
df = df[(z_scores < threshold).all(axis=1)] 

In [7]:
#standardizing the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
f[['Income ($)', 'Monthly Expenses ($)', 'Dependents', 'Job Stability (1-10)', 'Lifestyle Cost (1-10)']] = scaler.fit_transform(df[['Income ($)', 'Monthly Expenses ($)', 'Dependents', 'Job Stability (1-10)', 'Lifestyle Cost (1-10)']])

In [8]:
#save cleaned data
df.to_csv('cleaned_efData.csv', index=False)

# Model creation

In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

In [10]:
# Load your data
data = pd.read_csv('cleaned_efData.csv')

# Define features and target
X = data[['Income ($)', 'Monthly Expenses ($)', 'Dependents', 'Job Stability (1-10)', 'Lifestyle Cost (1-10)']]
y = data['Recommended Emergency Fund ($)']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
#randomforest
random_forest = RandomForestRegressor(n_estimators=100, random_state=42)
random_forest.fit(X_train, y_train)
y_pred_rf = random_forest.predict(X_test)


rmse_rf = mean_squared_error(y_test, y_pred_rf, squared=False)
print(rmse_rf)

c:\Users\eskim\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [22]:
print(rmse_rf)

3970.4124392360045


In [12]:
#saving the forest
import joblib
joblib.dump(random_forest, 'random_forest_model.pkl')

['random_forest_model.pkl']

In [21]:
# Load the saved model
loaded_model = joblib.load('random_forest_model.pkl')

# sample
user_input = {
    'Income ($)': 10000,
    'Monthly Expenses ($)':3000,
    'Dependents': 5,
    'Job Stability (1-10)': 1,
    'Lifestyle Cost (1-10)': 0
}

# Converting (assuming dict)
user_df = pd.DataFrame([user_input])

# Make a prediction
predicted_emergency_fund = loaded_model.predict(user_df)
print(f"Predicted Recommended Emergency Fund: ${predicted_emergency_fund[0]:,.2f}")

Predicted Recommended Emergency Fund: $66,899.98
